In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [4]:
!ls


datalab  drive


In [0]:
!pwd

/content


In [0]:
!pip install -q keras

In [0]:
!pip install -q keras_tqdm

In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Lambda, Input, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.optimizers import Adam, SGD, Adamax, Nadam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger, EarlyStopping
import keras.backend as K
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras_tqdm import TQDMNotebookCallback

import json
import os
import numpy as np
from PIL import Image,ImageDraw
import pandas as pd
import h5py
import math
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [8]:
image=np.load('drive/first_colab_project/project1/image.npy')
previous_state=np.load('drive/first_colab_project/project1/previous_state.npy')
label=np.load('drive/first_colab_project/project1/label.npy')
print image.shape
print previous_state.shape
print label.shape


(3035, 240, 352, 3)
(3035, 18)
(3035, 2)


In [0]:
adam = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

In [0]:
previous_statem=previous_state[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17]]

labelm=label[:,1]

In [16]:

image_input_shape = image.shape[1:]
state_input_shape = previous_statem.shape[1:]
activation = 'relu'
img_stack = Conv2D(50, (3, 3), name="convolution0", activation=activation)(pic_input)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(32, (3, 3), activation=activation, name='convolution1')(img_stack)
img_stack = MaxPooling2D(pool_size=(2, 2))(img_stack)
img_stack = Conv2D(40, (3, 3), activation=activation, name='convolution2')(img_stack)
img_stack = MaxPooling2D(pool_size=(2, 2))(img_stack)
img_stack = Conv2D(5, (3, 3), activation=activation, name='convolution3')(img_stack)
img_stack = MaxPooling2D(pool_size=(2, 2))(img_stack)
img_stack = Flatten()(img_stack)
img_stack = Dropout(0.2)(img_stack)

#Inject the state input


# Add a few dense layers to finish the model
img_stack = Dense(5, activation=activation, name='dense1')(img_stack)
state_input = Input(shape=state_input_shape)
merged = concatenate([img_stack, state_input])
merged = Dense(5, activation=activation, name='dense2')(merged)
merged = Dropout(0.1)(merged)
merged = Dense(1, name='output')(merged)

model = Model(inputs=[pic_input, state_input], outputs=merged)
# for layer in model.layers[:-10]:
#    layer.trainable = False
# for layer in model.layers[-10:]:
#    layer.trainable = True
model.compile(optimizer=adam, loss='mse')


#train
model.summary()




__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 240, 352, 3)  0                                            
__________________________________________________________________________________________________
convolution0 (Conv2D)           (None, 238, 350, 50) 1400        input_6[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_11 (MaxPooling2D) (None, 119, 175, 50) 0           convolution0[0][0]               
__________________________________________________________________________________________________
convolution1 (Conv2D)           (None, 117, 173, 32) 14432       max_pooling2d_11[0][0]           
__________________________________________________________________________________________________
max_poolin

In [0]:
def checkAndCreateDir(full_path):
    """Checks if a given path exists and if not, creates the needed directories.
            Inputs:
                full_path: path to be checked
    """
    if not os.path.exists(os.path.dirname(full_path)):
        try:
            os.makedirs(os.path.dirname(full_path))
        except OSError as exc:  # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [0]:

# #callbacks
plateau_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001, verbose=1)
checkpoint_filepath = os.path.join('drive/first_colab_project/', 'models3', '{0}_model.{1}-{2}.h5'.format('model', '{epoch:02d}', '{val_loss:.7f}'))
checkAndCreateDir(checkpoint_filepath)
checkpoint_callback = ModelCheckpoint(checkpoint_filepath, save_best_only=True, verbose=1)
csv_callback = CSVLogger(os.path.join('drive/first_colab_project/', 'training_log.csv'))
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=15 ,verbose=1)
callbacks=[plateau_callback, csv_callback, checkpoint_callback, early_stopping_callback]

In [0]:
!pip install -q ipywidgets
#jupyter nbextension enable --py --sys-prefix widgetsnbextension  # can be skipped for notebook version 5.3

In [25]:
history = model.fit(x=[image,previous_statem],y=labelm, epochs=50,batch_size=200,validation_split=0.15, verbose=1,callbacks=callbacks)

Train on 2579 samples, validate on 456 samples
Epoch 1/50
2579/2579 [==============================] - 22s 8ms/step - loss: 38586.1599 - val_loss: 36281.5223

Epoch 00001: val_loss improved from 39089.95069 to 36281.52227, saving model to drive/first_colab_project/models3/model_model.01-36281.5222725.h5
Epoch 2/50
2579/2579 [==============================] - 15s 6ms/step - loss: 36434.8178 - val_loss: 34449.9794

Epoch 00002: val_loss improved from 36281.52227 to 34449.97937, saving model to drive/first_colab_project/models3/model_model.02-34449.9793723.h5
Epoch 3/50
2579/2579 [==============================] - 15s 6ms/step - loss: 36428.3384 - val_loss: 34156.3096

Epoch 00003: val_loss improved from 34449.97937 to 34156.30955, saving model to drive/first_colab_project/models3/model_model.03-34156.3095532.h5
Epoch 4/50
2579/2579 [==============================] - 16s 6ms/step - loss: 39886.0531 - val_loss: 32936.7455

Epoch 00004: val_loss improved from 34156.30955 to 32936.74548, sav

2579/2579 [==============================] - 15s 6ms/step - loss: 31324.3708 - val_loss: 28663.9800

Epoch 00008: val_loss improved from 29949.78344 to 28663.98002, saving model to drive/first_colab_project/models3/model_model.08-28663.9800233.h5
Epoch 9/50
2579/2579 [==============================] - 16s 6ms/step - loss: 29900.2109 - val_loss: 27814.0418

Epoch 00009: val_loss improved from 28663.98002 to 27814.04180, saving model to drive/first_colab_project/models3/model_model.09-27814.0418037.h5
Epoch 10/50
2579/2579 [==============================] - 16s 6ms/step - loss: 32905.0392 - val_loss: 26804.6754

Epoch 00010: val_loss improved from 27814.04180 to 26804.67540, saving model to drive/first_colab_project/models3/model_model.10-26804.6754043.h5
Epoch 11/50
2579/2579 [==============================] - 16s 6ms/step - loss: 31309.4143 - val_loss: 25980.4366

Epoch 00011: val_loss improved from 26804.67540 to 25980.43661, saving model to drive/first_colab_project/models3/model_mod

2579/2579 [==============================] - 16s 6ms/step - loss: 25936.0659 - val_loss: 25122.9230

Epoch 00015: val_loss improved from 25323.51062 to 25122.92295, saving model to drive/first_colab_project/models3/model_model.15-25122.9229544.h5
Epoch 16/50
2579/2579 [==============================] - 16s 6ms/step - loss: 29071.5544 - val_loss: 25612.0134

Epoch 00016: val_loss did not improve from 25122.92295
Epoch 17/50
2579/2579 [==============================] - 16s 6ms/step - loss: 26320.2923 - val_loss: 26671.4982

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.000500000023749.

Epoch 00017: val_loss did not improve from 25122.92295
Epoch 18/50
2579/2579 [==============================] - 16s 6ms/step - loss: 25848.8221 - val_loss: 25273.1482

Epoch 00018: val_loss did not improve from 25122.92295
Epoch 19/50
2579/2579 [==============================] - 16s 6ms/step - loss: 26647.7393 - val_loss: 24885.5536

Epoch 00019: val_loss improved from 25122.92295 to 24885.55


Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.000250000011874.

Epoch 00022: val_loss did not improve from 24176.18902
Epoch 23/50
2579/2579 [==============================] - 16s 6ms/step - loss: 25300.4494 - val_loss: 24744.0994

Epoch 00023: val_loss did not improve from 24176.18902
Epoch 24/50
2579/2579 [==============================] - 16s 6ms/step - loss: 25636.5796 - val_loss: 24264.5950

Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.000125000005937.

Epoch 00024: val_loss did not improve from 24176.18902
Epoch 25/50
2579/2579 [==============================] - 16s 6ms/step - loss: 25846.9191 - val_loss: 24630.2152

Epoch 00025: val_loss did not improve from 24176.18902
Epoch 26/50
2579/2579 [==============================] - 16s 6ms/step - loss: 23497.2561 - val_loss: 24696.6279

Epoch 00026: ReduceLROnPlateau reducing learning rate to 6.25000029686e-05.

Epoch 00026: val_loss did not improve from 24176.18902
Epoch 27/50
2579/2579 [==================

2579/2579 [==============================] - 16s 6ms/step - loss: 24388.7654 - val_loss: 24355.0905

Epoch 00030: ReduceLROnPlateau reducing learning rate to 1.56250007421e-05.

Epoch 00030: val_loss did not improve from 24176.18902
Epoch 31/50
2579/2579 [==============================] - 16s 6ms/step - loss: 27548.6680 - val_loss: 24316.4135

Epoch 00031: val_loss did not improve from 24176.18902
Epoch 32/50
2579/2579 [==============================] - 16s 6ms/step - loss: 24337.2849 - val_loss: 24376.8553

Epoch 00032: ReduceLROnPlateau reducing learning rate to 7.81250037107e-06.

Epoch 00032: val_loss did not improve from 24176.18902
Epoch 33/50
2579/2579 [==============================] - 16s 6ms/step - loss: 27199.6559 - val_loss: 24405.8275

Epoch 00033: val_loss did not improve from 24176.18902
Epoch 34/50
2579/2579 [==============================] - 16s 6ms/step - loss: 25218.3732 - val_loss: 24444.9590

Epoch 00034: ReduceLROnPlateau reducing learning rate to 3.90625018554e-0